In [1]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


# AND Gate

In [33]:
T, F = 1.0, -1.0
bias = 1.0

In [34]:
training_input = [
    [T, T, bias],
    [T, F, bias],
    [F, T, bias],
    [F, F, bias],
]

training_output = [
    [T],
    [F],
    [F],
    [F]
]

In [35]:
w = tf.Variable(tf.random_normal([3, 1], dtype=tf.float32))

In [36]:
#    0_____
#_____|
def step(x):
    is_greater = tf.greater(x, 0)
    as_float = tf.to_float(is_greater)
    doubled = tf.multiply(as_float, 2)
    return tf.subtract(doubled, 1)

In [37]:
output = step(tf.matmul(training_input, w))
error = tf.subtract(training_output, output)
mse = tf.reduce_mean(tf.square(error))

In [38]:
delta = tf.matmul(training_input, error, transpose_a=True)
train = tf.assign(w, tf.add(w, delta))

In [39]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [40]:
err, target = 1, 0
epoch, max_epochs = 0, 10

In [41]:
while err > target and epoch < max_epochs:
    epoch += 1
    err, _ = sess.run([mse, train])
    print(sess.run([w]))

[array([[-1.1666737],
       [ 2.3665142],
       [-1.8197935]], dtype=float32)]
[array([[ 2.8333263],
       [ 2.3665142],
       [-1.8197935]], dtype=float32)]
[array([[ 2.8333263],
       [ 2.3665142],
       [-1.8197935]], dtype=float32)]


In [42]:
print('epoch', epoch, 'mse: ', err)

epoch 3 mse:  0.0


# CNN

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def create_new_conv_layer(input_data, num_input_channels,
                          num_filters, filter_shape, pool_shape,
                         name):
    conv_filt_shape = [filter_shape[0], filter_shape[1], 
                      num_input_channels, num_filters]
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape,stddev=0.03),
                         name=name + '_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')
    out_layer = tf.nn.conv2d(input_data, weights, 
                            [1, 1, 1, 1], padding='SAME')
    out_layer += bias
    out_layer = tf.nn.relu(out_layer)
    
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize,
                              strides=strides, padding='SAME')
    
    return out_layer

In [3]:
x = tf.placeholder(tf.float32, [None, 784])
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

In [4]:
layer1 = create_new_conv_layer(x_shaped, 1, 32, 
                               [5, 5], [2, 2], name='layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, 
                               [5, 5], [2, 2], name='layer2')

In [5]:
flattend = tf.reshape(layer2, [-1, 7 * 7 * 64])
wd1 = tf.Variable(tf.truncated_normal([7*7*64, 1000], stddev=0.03),
                 name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattend, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

In [6]:
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03),
                 name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01),
                 name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

In [7]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer2,
                                           labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
init_op = tf.global_variables_initializer()

In [9]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
y_train_one_hot = keras.utils.to_categorical(y_train)
y_test_one_hot = keras.utils.to_categorical(y_test)
epochs = 10
batch_size = 50

In [37]:
sess = tf.Session()
sess.run(init_op)
total_batch = int(len(x_test) / batch_size)
    
avg_cost = 0
for j in range(epochs):
    for i in range(total_batch):
        batch_x = x_train[i*batch_size:(i+1)*batch_size]
        batch_x = batch_x.reshape((batch_x.shape[0], np.prod(batch_x.shape[1:])))
        batch_y = y_train_one_hot[i*batch_size:(i+1)*batch_size]
        _, c = sess.run([optimizer, cross_entropy],
                        feed_dict={x: batch_x, y: batch_y})
        avg_cost = c / total_batch
        x_test_reshaped = x_test.reshape((x_test.shape[0], np.prod(x_test.shape[1:])))
        test_acc = sess.run(accuracy,
                            feed_dict={x: x_test_reshaped[0:batch_size],
                                         y:y_test_one_hot[0:batch_size]})
        if i % 20 == 0:
            print('avg_cost =', avg_cost, 'accuracy = %.8f' % test_acc)

avg_cost = 0.06868060111999512 accuracy = {0.18000001}
avg_cost = 0.006436520218849182 accuracy = {0.80000001}
avg_cost = 0.003877345323562622 accuracy = {0.88000000}
avg_cost = 0.001536543220281601 accuracy = {0.98000002}
avg_cost = 0.001218758299946785 accuracy = {0.94000000}
avg_cost = 0.0012671712040901184 accuracy = {0.95999998}
avg_cost = 0.0007749784737825394 accuracy = {0.95999998}
avg_cost = 0.0026937612891197206 accuracy = {0.95999998}
avg_cost = 0.0014674462378025054 accuracy = {0.94000000}
avg_cost = 0.0009921641647815704 accuracy = {0.92000002}
avg_cost = 0.001780131459236145 accuracy = {0.94000000}
avg_cost = 0.0016095146536827087 accuracy = {0.94000000}
avg_cost = 0.0006643108278512954 accuracy = {0.95999998}
avg_cost = 0.0007484102994203567 accuracy = {0.95999998}
avg_cost = 0.000568915456533432 accuracy = {0.95999998}
avg_cost = 0.0006020108610391617 accuracy = {0.95999998}
avg_cost = 0.00045878279954195025 accuracy = {0.95999998}
avg_cost = 0.0016750015318393707 accur

In [58]:
err = 0
for i in range(x_test_reshaped.shape[0]):
    x_single = x_test_reshaped[i]
    res = sess.run(y_, feed_dict={x: x_single[np.newaxis, :]})
    pred_y = res.argmax(-1)
    err += np.abs(pred_y - y_test[i])

In [63]:
saver = tf.train.Saver()
saver.save(sess, './cnn2conv')

'./cnn2conv'

In [65]:
sess.close()

In [66]:
saver = tf.train.import_meta_graph('./cnn2conv.meta')

In [69]:
tfs = tf.InteractiveSession()
saver.restore(tfs, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./cnn2conv


In [70]:
res = tfs.run(y_, feed_dict={x: x_test_reshaped[:200]})
print(res.argmax(-1))

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 5 3 7 4 6 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9 6 0 5 4 9 9 2 1 9 4 8
 7 3 9 7 4 4 4 9 2 5 4 7 6 7 9 0 5 8 5 6 6 5 7 8 1 0 1 6 4 6 7 3 1 7 1 8 2
 0 2 9 8 5 5 1 5 6 0 3 4 4 6 5 4 6 5 4 5 1 4 4 7 2 3 2 7 1 8 1 8 1 8 5 0 8
 9 2 5 0 1 1 1 0 9 0 3 1 6 4 2]
